In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install -q openai
!pip install -q tqdm
!pip install -q -U google-generativeai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Generación con OpenAI

In [ ]:
import openai
import os

# Configurar la clave de API de OpenAI
openai.api_key = 'XXXXXXXXXXXXXXXXXXXXXXX' # Anonimizado
os.environ["OPENAI_API_KEY"] = openai.api_key

openai.api_key = os.environ.get('OPENAI_API_KEY')
client = openai.OpenAI()
answer = client.chat.completions.create

In [ ]:
import pandas as pd
import json

n_msj = 100

def generate_dataset(output_file="generated_data.json"):

    # Definir el prompt para GPT-4
    PROMPT = f"""
          Debes generar {n_msj} datos para cada una de las 3 categorías relacionadas a procedimientos bancarios.
          USA las DEFINICIONES de clase como criterios clave para generar los mensajes.

          DEFINICIÓN PARA CADA TIPO DE MENSAJE:

          1.- Banca: Incluir preguntas relacionadas con operaciones bancarias comunes, como consultar saldos, generar cartolas bancarias, cambiar claves, realizar transferencias, abrir cuentas, entre otras.
          2.- Seguros: Crear preguntas relacionadas con seguros ofrecidos por bancos, como seguros para vehículos, seguros de vida o de hogar, seguros de viaje, de vida, de salud, entre otros; incluyendo cómo cotizarlos, contratarlos o cancelarlos.
          3.- Renuncia: Formular preguntas sobre cómo cerrar cuentas bancarias, cancelar servicios o renunciar a productos financieros.

          RECUERDA DEVOLVER {n_msj} MENSAJES PARA CADA CATAEGORÍA Y SOLO EN FORMATO JSON.
          """

    # Completar el texto utilizando el modelo de OpenAI
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.5,
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """Eres un asistente experto en servicios bancarios y de seguros. Tu tarea es generar un conjunto de datos ficticios sobre autogestión bancaria.
                                           Debes devolver un json con la siguiente estructura:
                                            {"Categoría": " ",
                                             "Pregunta": " ",
                                             "Respuesta": " "}"""},
            {"role": "user", "content": PROMPT}
        ]
      )

    # Obtener el mensaje generado
    gen_message = completion.choices[0].message.content.strip()

    print("Respuesta generada por la API:\n", gen_message)  # Depuración

    # Guardar el JSON bruto para inspección
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(gen_message)
        print(f"Archivo JSON generado y guardado en: {output_file}")

    try:
        # Intentar convertir el texto JSON generado en un objeto Python
        data = json.loads(gen_message)
        return data
    except json.JSONDecodeError as e:
        print(f"Error al procesar el JSON generado: {e}")
        return None

In [85]:
# Generar el JSON
generated_data = generate_dataset("dataset_ficticio.json")

Respuesta generada por la API:
 {
  "Banca": [
    {"Categoría": "Banca", "Pregunta": "¿Cómo puedo consultar el saldo de mi cuenta?", "Respuesta": "Puedes consultar el saldo de tu cuenta a través de la banca en línea, la aplicación móvil o en un cajero automático."},
    {"Categoría": "Banca", "Pregunta": "¿Qué necesito para abrir una cuenta bancaria?", "Respuesta": "Necesitas presentar una identificación válida, comprobante de domicilio y, en algunos casos, un depósito inicial."},
    {"Categoría": "Banca", "Pregunta": "¿Cómo cambio la clave de mi tarjeta de débito?", "Respuesta": "Puedes cambiar la clave de tu tarjeta de débito en un cajero automático o a través de la banca en línea."},
    {"Categoría": "Banca", "Pregunta": "¿Cuál es el límite de transferencia diaria?", "Respuesta": "El límite de transferencia diaria puede variar según el tipo de cuenta; verifica los detalles en tu banca en línea."},
    {"Categoría": "Banca", "Pregunta": "¿Puedo generar una cartola bancaria en líne

In [92]:
import pandas as pd
import json

# Cargar el JSON desde el archivo proporcionado
file_path = ('/content/dataset_ficticio.json')
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extraer los datos del JSON
datos = data.get("Seguros", [])

# Crear un DataFrame
df = pd.DataFrame(datos)
df.head(5)

,Categoría,Pregunta,Respuesta
0,Seguros,¿Cómo puedo contratar un seguro de automóvil?,Puedes contratar un seguro de automóvil a trav...
1,Seguros,¿Qué documentos necesito para contratar un seg...,"Necesitarás tu identificación, comprobante de ..."
2,Seguros,¿Cómo puedo cancelar mi seguro de hogar?,Debes comunicarte con el servicio al cliente p...
3,Seguros,¿Qué cubre un seguro de viaje?,"Un seguro de viaje puede cubrir cancelaciones,..."
4,Seguros,¿Cómo puedo obtener una cotización para un seg...,Puedes obtener una cotización para un seguro d...


In [93]:
df.to_excel('/content/drive/MyDrive/Big Data/Práctica 2/Generación de datos/dataset_ficticio3_seguros.xlsx', index=False)

# Generación con Gemini

In [ ]:
from google.colab import userdata
import os
import google.generativeai as genai

# Or use `os.getenv('API_KEY')` to fetch an environment variable.
API_KEY=userdata.get('API_KEY_GEMINI')

genai.configure(api_key=API_KEY)

In [ ]:
import json
import google.generativeai as genai

def query_gemini(output_file=None):
    """
    Consulta a la API de Gemini con un prompt y devuelve la respuesta como JSON.
    También guarda la información generada en un archivo JSON si se especifica.

    Args:
        prompt (str): El prompt que se envía al modelo.
        output_file (str): Ruta del archivo para guardar la respuesta generada.

    Returns:
        dict: Respuesta procesada en formato JSON, o None si ocurre un error.
    """

    # Prompt para generar datos ficticios
    prompt = """
    Eres un asistente experto en servicios bancarios y de seguros. Tu tarea es generar un conjunto de datos ficticios sobre autogestión bancaria.
    Debes generar un archivo .json con 50 mensajes para cada una de las 3 categorías relacionadas a procedimientos bancarios.

    USA las DEFINICIONES de clase como criterios clave para generar los mensajes.
    DEFINICIÓN PARA CADA TIPO DE MENSAJE:

    1.- Banca: Incluir preguntas relacionadas con operaciones bancarias comunes, como consultar saldos, generar cartolas bancarias, cambiar claves, realizar transferencias o abrir cuentas.
    2.- Seguros: Crear preguntas relacionadas con seguros ofrecidos por bancos, como seguros para vehículos, seguros de vida o de hogar, incluyendo cómo cotizarlos, contratarlos o cancelarlos.
    3.- Renuncia: Formular preguntas sobre cómo cerrar cuentas bancarias, cancelar servicios o renunciar a productos financieros.

    Debes devolver un json con la siguiente estructura:
      {"Categoría": " ",
        "Pregunta": " ",
        "Respuesta": " "}

    RECUERDA DEVOLVER SOLO EL FORMATO JSON ANTERIOR.
    """

    try:
        # Configurar el modelo de Gemini
        model = genai.GenerativeModel(
            "gemini-1.5-flash-latest",
            generation_config={"temperature": 0.5, # Modificable según variabilidad deseada del dato
                               "response_mime_type": "application/json"}
        )

        # Enviar el prompt a la API
        raw_response = model.generate_content(prompt)

        # Extraer el texto de la respuesta
        response_text = raw_response.text.strip()

        # Verificar si la respuesta contiene múltiples JSON concatenados
        json_objects = []
        for line in response_text.splitlines():
            try:
                json_objects.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error al procesar una línea: {line}")
                print(f"Detalle del error: {e}")

        # Guardar la lista de JSONs en un archivo si se proporciona un nombre de archivo
        if output_file:
            with open(output_file, "w", encoding="utf-8") as file:
                json.dump(json_objects, file, indent=2, ensure_ascii=False)
                print(f"Archivo JSON generado y guardado en: {output_file}")

        return json_objects

    except Exception as e:
        print(f"Error al comunicarse con la API de Gemini: {e}")
        return None


In [184]:
# Consultar a Gemini y guardar la respuesta en un archivo
response = query_gemini("data_gemini.json")

# Validar y mostrar la respuesta
if response:
  print("Respuesta generada con éxito:")
  print(json.dumps(response, indent=2, ensure_ascii=False))
else:
  print("No se pudo generar una respuesta válida.")

Archivo JSON generado y guardado en: data_gemini.json
Respuesta generada con éxito:
[
  {
    "dataset": [
      {
        "Categoría": "Banca",
        "Pregunta": "¿Cómo puedo consultar el saldo de mi cuenta de ahorros?",
        "Respuesta": "Para consultar el saldo de su cuenta de ahorros, puede acceder a nuestra aplicación móvil, nuestra página web o llamar a nuestro servicio de atención al cliente."
      },
      {
        "Categoría": "Banca",
        "Pregunta": "¿Cómo puedo generar una cartola bancaria de los últimos tres meses?",
        "Respuesta": "Puede generar su cartola bancaria de los últimos tres meses a través de nuestra aplicación móvil, nuestra página web o solicitándola a nuestro servicio de atención al cliente."
      },
      {
        "Categoría": "Banca",
        "Pregunta": "¿Cómo puedo cambiar mi clave de acceso a la banca online?",
        "Respuesta": "Para cambiar su clave de acceso, ingrese a nuestra página web, acceda a la sección de configuración de s

In [181]:
import pandas as pd
import json

# Cargar el JSON desde el archivo proporcionado
file_path = ('/content/data_gemini.json')
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Extraer los datos del JSON
datos = data[0].get("data", [])

# Crear un DataFrame
df = pd.DataFrame(datos)
df

,Categoría,Pregunta,Respuesta
0,Banca,¿Cómo puedo consultar el saldo de mi cuenta de...,Para consultar el saldo de su cuenta de ahorro...
1,Banca,¿Cómo puedo generar una cartola bancaria de lo...,Puede generar su cartola bancaria de los últim...
2,Banca,¿Cómo cambio mi clave de acceso a la banca onl...,"Para cambiar su clave, acceda a la sección 'Se..."
3,Banca,¿Cómo puedo realizar una transferencia a otra ...,"Para realizar una transferencia, ingrese a nue..."
4,Banca,¿Cómo puedo abrir una nueva cuenta corriente?,"Para abrir una cuenta corriente, puede visitar..."
5,Banca,¿Qué comisiones se aplican a mi cuenta de cheq...,Las comisiones aplicables a su cuenta de chequ...
6,Banca,¿Cómo puedo depositar un cheque en mi cuenta?,Puede depositar un cheque en su cuenta a travé...
7,Banca,¿Cómo puedo solicitar una tarjeta de débito?,Puede solicitar una tarjeta de débito a través...
8,Banca,¿Cómo puedo bloquear mi tarjeta de débito en c...,Puede bloquear su tarjeta de débito llamando a...
9,Banca,¿Cómo puedo acceder a mi historial de transacc...,Puede acceder a su historial de transacciones ...


In [ ]:
df.to_excel('/content/drive/MyDrive/Big Data/Práctica 2/Generación de datos/dataset_ficticios_gemini.xlsx')